#### Notebook Parameters

In [ ]:
INPUT_PATH = '../input/'

# Import Libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.utils import np_utils
from keras.layers.core import Dense, Dropout
from keras.layers import Convolution2D, MaxPooling2D, Flatten
from keras.utils import plot_model, np_utils
from sklearn.model_selection import train_test_split

# Read Data
***
## Check files

In [ ]:
from subprocess import check_output
print(check_output(["ls", INPUT_PATH]).decode("utf8"))

## Read Data

In [ ]:
train = pd.read_csv(INPUT_PATH + 'train.csv')
train.sample(10)

# Prepare Labels
***
Convert labels to one-hot encoded

In [ ]:
train_y = keras.utils.to_categorical(train.label.values)

# Convert Train
***
Convert train to a numpy array

In [ ]:
train_x = train.drop('label', axis=1).values

## Normalize grayscale
***
Convert values from [0...255] to [0...1].
NN converge faster on values of 0 to 1

In [ ]:
def normalize(array):
    # Normalize the data
    return array.astype(np.float32) / 255.0

train_x = normalize(train_x)

# Convert to 2 dimensional


In [ ]:
train_x = train_x.reshape(train_x.shape[0],28,28,1)

# Split train and test

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size=0.33, random_state=777)

# Create Keras Model

In [ ]:
def get_model():
    model = keras.models.Sequential()
    # Hidden layer of 512 neurons
    model.add(Convolution2D(32,3,3, activation='relu', input_shape=(28,28,1,)))
    model.add(Convolution2D(32,3,3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(10, activation='softmax'))
    return model

## Compile the model


In [ ]:
model = get_model()
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

## Fit the model with train values

In [ ]:
model.fit(train_x, train_y, batch_size=32, epochs=10, validation_data=(test_x, test_y))

In [ ]:
score = model.evaluate(test_x, test_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Last Best
```
model.add(Convolution2D(32,3,3, activation='relu', input_shape=(28,28,1,)))
model.add(Convolution2D(32,3,3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10, activation='softmax'))**

EPOCHS = 10

Test loss: 0.0659512992929
Test accuracy: 0.984925164208
```

# Predict and Submit
***

## Train with entire dataset

In [ ]:
train = pd.read_csv(INPUT_PATH + 'train.csv')
train_y = keras.utils.to_categorical(train.label.values)
train_x = train.drop('label', axis=1).values
train_x = normalize(train_x)
train_x = train_x.reshape(train_x.shape[0],28,28,1)
model = get_model()
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.fit(train_x, train_y, batch_size=32, epochs=10)

## Predict

In [ ]:
test = pd.read_csv(INPUT_PATH + 'test.csv')
test_x = normalize(test.values)
test_x = test_x.reshape(test_x.shape[0],28,28,1)

predictions = model.predict(test_x)

# Generate submission 

In [ ]:
import time

# select the indix with the maximum probability
predictions = np.argmax(predictions,axis = 1)
predictions = pd.Series(predictions,name="Label")

submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),predictions],axis = 1)

# Creo una variable con la fecha y hora
datetime = time.strftime("%Y%m%d_%H%M%S")

# Creo el archivo de submission con la fecha actual en el nombre del archivo.
submission.to_csv('submission.' + datetime + '_loss_' + str(score[0]) + '_acc_' + str(score[1]) + '.csv', index=False)